In [1]:
import numpy as np
import json
from scipy.stats import gaussian_kde

import sys
sys.path.append('./../figures')
from drawChiEff import drawChieffs
sys.path.append('./../code/emceeCode')
from posterior_helper_functions import *

In [2]:
# Helper function to calculate and print out quantiles
def print_quantiles(arr, low, high): 
    median = np.quantile(arr, 0.5)
    upper = np.quantile(arr, high)
    lower = np.quantile(arr, low)
    print(f'{np.round(median, 3)} + {np.round(upper - median, 3)} / - {np.round(median - lower, 3)}')
    return median,lower,upper

# Helper function to draw numbers uniformly between 'low' and 'high'
def draw_uniform(n, low, high): 
    return np.random.random(n)*(high-low)+low

In [3]:
# Load data
root = '../data'
models = ['betaPlusMixture', 'betaPlusTruncatedMixture', 'betaSpikePlusMixture', 'betaSpikePlusTruncatedMixture']

eps_min = 0.025

data_dict = {}
for model in models: 
    with open(f'{root}/component_spin_{model}.json', 'r') as f:
        data = json.load(f)
    if model=='betaSpikePlusMixture' or model=='betaSpikePlusTruncatedMixture':
        data_dict[model]= {key:np.asarray(data[key]['processed'])[np.asarray(data['sigma_spike']['processed'])>eps_min] for key in data.keys()}
    else:
        data_dict[model]= {key:np.asarray(data[key]['processed']) for key in data.keys()}

### Calculate all macros and store them in a dictionary

In [4]:
# Dict to store all the macros
macros_dict = {}
for model in models: 
    macros_dict[model] = {}

Fraction in spike

In [5]:
for model in ['betaSpikePlusMixture', 'betaSpikePlusTruncatedMixture']: 
    
    print(model)
    frac_data = data_dict[model]['frac_in_spike']
    median_f,lower_f,upper_f=print_quantiles(frac_data, 0, 0.9)
    
    macros_dict[model]['f_spike_median'] = median_f
    macros_dict[model]['f_spike_upper'] = upper_f
    macros_dict[model]['f_spike_lower'] = lower_f
    
    print()

betaSpikePlusMixture
0.217 + 0.308 / - 0.217

betaSpikePlusTruncatedMixture
0.332 + 0.274 / - 0.331



$z_\mathrm{min}$

In [6]:
for model in ['betaPlusTruncatedMixture', 'betaSpikePlusTruncatedMixture']: 
    
    print(model)
    zmin_data = data_dict[model]['cost_min']
    median_zmin,lower_zmin,upper_zmin = print_quantiles(zmin_data, 0.05, 0.95)
    
    macros_dict[model]['zmin_data_median'] = median_zmin
    macros_dict[model]['zmin_data_upper'] = upper_zmin
    macros_dict[model]['zmin_data_lower'] = lower_zmin
    print()

betaPlusTruncatedMixture
-0.549 + 0.188 / - 0.224

betaSpikePlusTruncatedMixture
-0.492 + 0.265 / - 0.26



$z_{1\%}$

In [7]:
# models without zmin
for model in ['betaPlusMixture', 'betaSpikePlusMixture']:
    
    MFs = data_dict[model]['MF_cost']
    sigma_tilts = data_dict[model]['sigma_cost']
    
    # calculate z1 for each hyperparameter
    n_samps = len(MFs)
    z1s = np.zeros(n_samps)
    
    for i,MF,sigma_tilt in zip(np.arange(n_samps), MFs, sigma_tilts): 
        
        costhetas_uniform = draw_uniform(10000,-1,1)                            
        weights = calculate_Gaussian_Mixture_1D(costhetas_uniform, 1, sigma_tilt, MF, -1, 1)
        costhetas = np.random.choice(costhetas_uniform, p=weights/np.sum(weights), size=1000)                               
        z1s[i] = np.quantile(costhetas, 0.01)
    
    # find quantiles on z1                                 
    print(model)
    median_z1,lower_z1,upper_z1 = print_quantiles(z1s, 0.05, 0.95)
    macros_dict[model]['z1_median'] = median_z1
    macros_dict[model]['z1_upper'] = upper_z1
    macros_dict[model]['z1_lower'] = lower_z1
    print()

# models with zmin
for model in ['betaPlusTruncatedMixture', 'betaSpikePlusTruncatedMixture']: 
    
    MFs = data_dict[model]['MF_cost']
    sigma_tilts = data_dict[model]['sigma_cost']
    zmins = data_dict[model]['cost_min']
    
    # calculate z1 for each hyperparameter
    n_samps = len(MFs)
    z1s = np.zeros(n_samps)
    
    for i,MF,sigma_tilt,zmin in zip(np.arange(n_samps), MFs, sigma_tilts,zmins):
        
        costhetas_uniform = draw_uniform(10000,zmin,1)                                  
        weights = calculate_Gaussian_Mixture_1D(costhetas_uniform, 1, sigma_tilt, MF, zmin, 1)
        costhetas = np.random.choice(costhetas_uniform, p=weights/np.sum(weights), size=1000)                               
        z1s[i] = np.quantile(costhetas, 0.01)
    
   # find quantiles on z1                                  
    print(model)
    median_z1,lower_z1,upper_z1=print_quantiles(z1s, 0.05, 0.95)
    macros_dict[model]['z1_median'] = median_z1
    macros_dict[model]['z1_upper'] = upper_z1
    macros_dict[model]['z1_lower'] = lower_z1
    print()

betaPlusMixture
-0.96 + 0.071 / - 0.021

betaSpikePlusMixture
-0.957 + 0.078 / - 0.025

betaPlusTruncatedMixture
-0.53 + 0.184 / - 0.214

betaSpikePlusTruncatedMixture
-0.473 + 0.267 / - 0.252



$\chi_{1\%}$ and $\chi_{99\%}$

In [8]:
# models without spike
for model in ['betaPlusMixture', 'betaPlusTruncatedMixture']:
    
    mu_chis = data_dict[model]['mu_chi']
    sigma_chis = data_dict[model]['sigma_chi']
    
    # calculate chi1 and chi99 for each hyperparameter
    n_samps = len(mu_chis)
    chi1s = np.zeros(n_samps)
    chi99s = np.zeros(n_samps)
    
    for i,mu_chi,sigma_chi in zip(np.arange(n_samps), mu_chis, sigma_chis): 
        
        chis_uniform = draw_uniform(10000,0,1)                  
        a,b = mu_sigma2_to_a_b(mu_chi, sigma_chi**2)   
        chis_uniform = draw_uniform(10000,0,1)                     
        weights = betaDistribution(chis_uniform, a,b)
        chis = np.random.choice(chis_uniform, p=weights/np.sum(weights), size=1000)                               
        chi1s[i] = np.quantile(chis, 0.01)
        chi99s[i] = np.quantile(chis, 0.99)
        
    # find quantiles on chi10 and chi90                                  
    print(model)
    print('chi_1:')
    median_chi1,lower_chi1,upper_chi1=print_quantiles(chi1s, 0.00, 0.90)
    macros_dict[model]['chi1_median'] = median_chi1
    macros_dict[model]['chi1_upper'] = upper_chi1
    macros_dict[model]['chi1_lower'] = lower_chi1
    print('chi_99:')
    median_chi99,lower_chi99,upper_chi99=print_quantiles(chi99s, 0.05, 0.95)
    macros_dict[model]['chi99_median'] = median_chi99
    macros_dict[model]['chi99_upper'] = upper_chi99
    macros_dict[model]['chi99_lower'] = lower_chi99
    print()

# models with spike
for model in ['betaSpikePlusMixture', 'betaSpikePlusTruncatedMixture']: 
    
    mu_chis = data_dict[model]['mu_chi']
    sigma_chis = data_dict[model]['sigma_chi']
    f_spikes = data_dict[model]['frac_in_spike']
    sigma_spikes = data_dict[model]['sigma_spike']
    
    # calculate chi1 and chi99 for each hyperparameter
    n_samps = len(mu_chis)
    chi1s = np.zeros(n_samps)
    chi99s = np.zeros(n_samps)
    
    for i,mu_chi,sigma_chi,f_spike,sigma_spike in zip(np.arange(n_samps), mu_chis,sigma_chis,f_spikes,sigma_spikes): 
        
        chis_uniform = draw_uniform(10000,0,1)                  
        a,b = mu_sigma2_to_a_b(mu_chi, sigma_chi**2)   
        weights = betaDistributionPlusSpike(chis_uniform, a, b, f_spike, sigma_spike)
        chis = np.random.choice(chis_uniform, p=weights/np.sum(weights), size=1000)                               
        chi1s[i] = np.quantile(chis, 0.01)
        chi99s[i] = np.quantile(chis, 0.99)
        
    # find quantiles on chi10 and chi90                                  
    print(model)
    print('chi_1:')
    median_chi1,lower_chi1,upper_chi1=print_quantiles(chi1s, 0.0, 0.90)
    macros_dict[model]['chi1_median'] = median_chi1
    macros_dict[model]['chi1_upper'] = upper_chi1
    macros_dict[model]['chi1_lower'] = lower_chi1
    print('chi_99:')
    median_chi99,lower_chi99,upper_chi99=print_quantiles(chi99s, 0.05, 0.95)
    macros_dict[model]['chi99_median'] = median_chi99
    macros_dict[model]['chi99_upper'] = upper_chi99
    macros_dict[model]['chi99_lower'] = lower_chi99
    print() 

betaPlusMixture
chi_1:
0.022 + 0.05 / - 0.021
chi_99:
0.581 + 0.183 / - 0.159

betaPlusTruncatedMixture
chi_1:
0.026 + 0.051 / - 0.025
chi_99:
0.565 + 0.188 / - 0.142

betaSpikePlusMixture
chi_1:
0.004 + 0.01 / - 0.003
chi_99:
0.643 + 0.205 / - 0.191

betaSpikePlusTruncatedMixture
chi_1:
0.003 + 0.008 / - 0.002
chi_99:
0.673 + 0.201 / - 0.209



$\chi_\mathrm{eff,1\%}$ and $\chi_\mathrm{eff,99\%}$ 

In [9]:
for model in models:
    
    nsamps = len(data_dict[model]['mu_chi'])
    
    chieff1s = np.zeros(nsamps)
    chieff99s = np.zeros(nsamps)
    
    # cycle through hyper parameters  
    for i in np.arange(nsamps): 

        mu_chi = data_dict[model]['mu_chi'][i]
        sigma_chi = data_dict[model]['sigma_chi'][i]                                
        MF_cost = data_dict[model]['MF_cost'][i]
        sigma_cost = data_dict[model]['sigma_cost'][i]
        Bq = data_dict[model]['Bq'][i]

        a, b = mu_sigma2_to_a_b(mu_chi, sigma_chi**2)

        # specific distribution we calculate depends on the model of interest
        if model=='betaPlusMixture': 
            f_spike = None
            sigma_spike = None
            zmin = -1

        elif model=='betaPlusTruncatedMixture': 
            f_spike = None
            sigma_spike = None
            zmin = data_dict[model]['cost_min'][i]

        elif model=='betaSpikePlusMixture': 
            f_spike = data_dict[model]['frac_in_spike'][i]
            sigma_spike = data_dict[model]['sigma_spike'][i]
            zmin = -1

        elif model=='betaSpikePlusTruncatedMixture':
            f_spike = data_dict[model]['frac_in_spike'][i]
            sigma_spike = data_dict[model]['sigma_spike'][i]
            zmin = data_dict[model]['cost_min'][i]

        else: 
            print('model not found, something wrong')
            break    

        # draw chi-effectives from this distribution
        chi_effs = drawChieffs(mu_chi, sigma_chi, MF_cost, sigma_cost, f_spike, sigma_spike, zmin, Bq, n=1000)
        chieff1s[i] = np.quantile(chi_effs, 0.01)
        chieff99s[i] = np.quantile(chi_effs, 0.99)

    # find quantiles on chieff1 and chieff99                                  
    print(model)
    print('chieff_1:')
    median_chieff1,lower_chieff1,upper_chieff1=print_quantiles(chieff1s, 0.05, 0.95)
    macros_dict[model]['chieff1_median'] = median_chieff1
    macros_dict[model]['chieff1_upper'] = upper_chieff1
    macros_dict[model]['chieff1_lower'] = lower_chieff1
    print('chieff_99:')
    median_chieff99,lower_chieff99,upper_chieff99=print_quantiles(chieff99s, 0.05, 0.95)
    macros_dict[model]['chieff99_median'] = median_chieff99
    macros_dict[model]['chieff99_upper'] = upper_chieff99
    macros_dict[model]['chieff99_lower'] = lower_chieff99
    print() 

/home/simona.miller/gwtc3-spin-studies/macros/./../code/emceeCode/posterior_helper_functions.py:257: RuntimeWarning: overflow encountered in exp
  f = np.exp(deltaM/m + deltaM/(m-deltaM))


betaPlusMixture
chieff_1:
-0.205 + 0.098 / - 0.129
chieff_99:
0.273 + 0.123 / - 0.087

betaPlusTruncatedMixture
chieff_1:
-0.1 + 0.065 / - 0.103
chieff_99:
0.267 + 0.122 / - 0.086

betaSpikePlusMixture
chieff_1:
-0.215 + 0.104 / - 0.132
chieff_99:
0.298 + 0.131 / - 0.103

betaSpikePlusTruncatedMixture
chieff_1:
-0.1 + 0.081 / - 0.108
chieff_99:
0.304 + 0.147 / - 0.099



### Make `.txt` file with macros

In [10]:
params = ['f_spike', 'zmin_data', 'z1', 'chi1', 'chi99', 'chieff1', 'chieff99'] 

with open('table1_macros.txt','w') as macrofile:
    
    for model in models: 
        print(model)
        for param in params: 
            
            try:
                median = macros_dict[model][f'{param}_median']
                upper = macros_dict[model][f'{param}_upper']
                lower = macros_dict[model][f'{param}_lower']

                upper_ebar = upper - median
                lower_ebar = median - lower
                
                macroname = model.replace('_','')+param.replace('_','').replace('1','One').replace('99','NinetyNine')
                
                if f'{median:.2f}'=='0.00': 
                    line = f"\\newcommand{{\\{macroname}}}{{{median:.3f}^{{+{upper_ebar:.3f}}}_{{-{lower_ebar:.3f}}}}}"
                else: 
                    line = f"\\newcommand{{\\{macroname}}}{{{median:.2f}^{{+{upper_ebar:.2f}}}_{{-{lower_ebar:.2f}}}}}"
                print(line)
                macrofile.write(line)
                macrofile.write("\n")
            except: 
                print(f'{param} not in {model}')
                
        print('')

betaPlusMixture
f_spike not in betaPlusMixture
zmin_data not in betaPlusMixture
\newcommand{\betaPlusMixturezOne}{-0.96^{+0.07}_{-0.02}}
\newcommand{\betaPlusMixturechiOne}{0.02^{+0.05}_{-0.02}}
\newcommand{\betaPlusMixturechiNinetyNine}{0.58^{+0.18}_{-0.16}}
\newcommand{\betaPlusMixturechieffOne}{-0.20^{+0.10}_{-0.13}}
\newcommand{\betaPlusMixturechieffNinetyNine}{0.27^{+0.12}_{-0.09}}

betaPlusTruncatedMixture
f_spike not in betaPlusTruncatedMixture
\newcommand{\betaPlusTruncatedMixturezmindata}{-0.55^{+0.19}_{-0.22}}
\newcommand{\betaPlusTruncatedMixturezOne}{-0.53^{+0.18}_{-0.21}}
\newcommand{\betaPlusTruncatedMixturechiOne}{0.03^{+0.05}_{-0.03}}
\newcommand{\betaPlusTruncatedMixturechiNinetyNine}{0.57^{+0.19}_{-0.14}}
\newcommand{\betaPlusTruncatedMixturechieffOne}{-0.10^{+0.06}_{-0.10}}
\newcommand{\betaPlusTruncatedMixturechieffNinetyNine}{0.27^{+0.12}_{-0.09}}

betaSpikePlusMixture
\newcommand{\betaSpikePlusMixturefspike}{0.22^{+0.31}_{-0.22}}
zmin_data not in betaSpikePlusMixt